In [6]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from datetime import datetime
import pandas as pd
import numpy as np
import warnings
import joblib

# 경고 제거
warnings.filterwarnings('ignore')

In [7]:
# 데이터 읽어오기
file_path = '.'
df = pd.read_excel(f'{file_path}/tb_stock.xlsx')

In [8]:
def train_and_save_model(df, model_filename='regression_bitcoin_xgboost.pkl'):
    stock_dataset = df[['sc_date', 'sc_coin']]
    stock_dataset['sc_date'] = pd.to_datetime(stock_dataset['sc_date'])
    stock_dataset.rename(columns={'sc_date': 'Date', 'sc_coin': 'stock'}, inplace=True)
    stock_dataset.set_index('Date', inplace=True)

    # 전체 기간 데이터를 사용하여 학습 데이터 준비
    days_in = min(5, len(stock_dataset) - 1)

    raw = []

    # 특성 생성: 과거 days_in일 기준으로 이동
    for i in range(min(days_in, len(stock_dataset)), 0, -1):
        raw.append(stock_dataset.shift(i))

    sum_df = pd.concat(raw, axis=1)
    sum_df.dropna(inplace=True)

    # 데이터 분리
    train = sum_df.values
    X = train[:, :-1]
    Y = train[:, -1]

    print(f'Dataset size: {len(X)} samples')  # 데이터셋 크기 확인

    if len(X) < 2:
        print("Not enough data to train the model")
        return

    # 하이퍼파라미터 그리드 설정
    param_grid = {
        'n_estimators': [50],
        'max_depth': [3],
        'learning_rate': [0.3, 0.5],
        'subsample': [1.0, 1.2],
        'colsample_bytree': [0.8],
        'gamma': [0],
        'min_child_weight': [1],
        'reg_alpha': [0],
        'reg_lambda': [2, 3]
    }

    # 모델 초기화
    model = XGBRegressor(objective='reg:squarederror', n_jobs=1)

    # GridSearchCV 초기화
    cv_value = min(3, len(X))  # cv는 최소 2 이상의 값을 가져야 함
    if cv_value < 2:
        print("Not enough data to perform cross-validation")
        return

    grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                            cv=cv_value, scoring='neg_mean_squared_error', verbose=1)

    # 모델 학습 및 최적의 하이퍼파라미터 탐색
    grid_search.fit(X, Y)

    # 최적의 하이퍼파라미터로 모델 생성
    best_model = grid_search.best_estimator_

    print(f'Best hyperparameters: {grid_search.best_params_}')

    # 최적화된 모델 저장
    joblib.dump(best_model, model_filename)
    print(f'Model saved as {model_filename}')

# 사용 예시:
df = pd.read_excel('./tb_stock.xlsx')  # 데이터프레임 로드
train_and_save_model(df)  # 모델 학습 및 저장


Dataset size: 3629 samples
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best hyperparameters: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.3, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 50, 'reg_alpha': 0, 'reg_lambda': 3, 'subsample': 1.0}
Model saved as regression_bitcoin_xgboost.pkl


In [31]:
def xgboost_predict(stock, df, day, model_filename):
    # 데이터 전처리
    stock_dataset = df[['sc_date', stock]]
    stock_dataset['sc_date'] = pd.to_datetime(stock_dataset['sc_date'])
    stock_dataset.rename(columns={'sc_date': 'Date', stock: 'stock'}, inplace=True)
    stock_dataset.set_index('Date', inplace=True)
    
    # 날짜 변환 및 계산
    today = datetime.today()
    month_format = str(today.month).zfill(2)  # 월 형식 정리
    day_format = str(today.day).zfill(2)      # 일 형식 정리
    today_format = datetime.strptime(f'{today.year}{month_format}{day_format}', '%Y%m%d')
    formatted_date = datetime.strptime(str(day), '%Y%m%d')

    start_day = 300
    end_day = (formatted_date - today_format).days + 1

    # 데이터 준비
    days_in = start_day  # 학습에 사용할 데이터 수
    day_out = 1          # 예측할 일 수
    raw = []

    # 특성 생성: 과거 days_in일과 미래 day_out일을 기준으로 이동
    for i in range(days_in, 0, -1):
        raw.append(stock_dataset.shift(i))
    for i in range(0, day_out):
        raw.append(stock_dataset.shift(-i))

    sum_df = pd.concat(raw, axis=1)
    sum_df.dropna(inplace=True)

    # 데이터 분리
    train = sum_df.values
    X = train[:, :-1]
    Y = train[:, -1]

    # 저장된 모델 로드
    model = joblib.load(model_filename)
    print(f'Model loaded from {model_filename}')

    # 예측 데이터 준비
    data_in = stock_dataset[-(days_in):]  # 예측을 위한 마지막 days_in일 데이터 슬라이싱
    data_in = data_in.values              # NumPy 배열로 변환
    data_in = data_in.reshape(1, -1)      # (1, n_features) 형태로 변환

    forecast_days = end_day  # 예측할 전체 값
    forecast_results = []

    # 반복하여 예측 수행
    for _ in range(forecast_days):
        # 예측 수행
        prediction = model.predict(data_in)
        forecast_results.append(prediction[0])

        # 데이터 업데이트: 예측 값을 데이터에 추가하고 가장 오래된 데이터를 제거
        new_data = np.append(data_in[0][1:], prediction)
        data_in = new_data.reshape(1, -1)

    # 예측 결과를 DataFrame으로 변환
    forecast_dates = pd.date_range(start=stock_dataset.index[-1],
                                periods=forecast_days)
    forecast_df = pd.DataFrame(forecast_results, index=forecast_dates, columns=['Forecast'])

    # 예측 결과 출력
    print('예측 일:', str(forecast_df.index[-1])[:10], ' 예측 값:', forecast_df.iloc[-1].values[0])

# 사용 예시:
# 예측할 날짜를 지정하고, 모델 파일 경로를 설정합니다.
df = pd.read_excel('./tb_stock.xlsx')  # 실제 데이터로 교체하세요.
xgboost_predict('sc_ss_stock', df, '20240828', model_filename='regression_samsung_xgboost_model.pkl')
xgboost_predict('sc_ap_stock', df, '20240828', model_filename='regression_apple_xgboost_model.pkl')
xgboost_predict('sc_coin', df, '20240828', model_filename='regression_bitcoin_xgboost_model.pkl')

Model loaded from regression_samsung_xgboost_model.pkl
예측 일: 2024-08-28  예측 값: 75031.77
Model loaded from regression_apple_xgboost_model.pkl
예측 일: 2024-08-28  예측 값: 227.01567
Model loaded from regression_bitcoin_xgboost_model.pkl
예측 일: 2024-08-28  예측 값: 58385.01
